In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
from tqdm import tqdm_notebook
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [2]:
retain_fields = ['date', 'case_number', 'employer', 'tot_foreign_positions', 'naic_code', 'job_title', 'job_code',
                 'soc_name', 'approval_status', 'annual_wage', 'annual_prevailing_wage', 'city',
                 'state', 'county', 'year']

# Fax data 02-06

We noted that there's no available fax data in 2007, so we load the fax data between 2002 and 2006:

In [3]:
for year in tqdm_notebook(range(2002, 2007)):
    year_str = str(year)[-2:]
    globals()['fax_'+year_str] = pd.read_csv('dataset/h1b_Fax_FY'+year_str+'.txt')

- For fax data, the fields `date`, `tot_foreign_positions`, `naic_code`, `soc_name`, `county` are missing, so we set the values of these fields to be `np.nan`

- The fields `case_number`, `employer`, `job_title`, `job_code`, `approval_status`, `city`, `state` can be directly obtained from the raw data, we just need to modify their names

- The fields `annual_wage`, `annual_prevailing_wage` need to be calculated using wages and units. So, we need to convert them from string-type to numerical-type using function `str2num` and `rp2num`

- We add a column `year` to represent the year of data source


In [4]:
def str2num(x):
    '''
    change string-type number to float
    '''
    try:
        return float(x)
    except:
        x = x.replace(',', '')
        if x[0] == '$':
            x = x[1:]
        try:
            return float(x)
        except:
            return np.nan


def rp2num(x):
    '''
    change RatePer to frequency number
    '''
    if x in ['Y', 'y']:
        return 1
    elif x in ['M', 'm']:
        return 12
    elif x in ['B', 'b']:
        return 26
    elif x in ['W', 'w']:
        return 52
    elif x in ['H', 'h']:
        return 2080
    else:
        return np.nan


def process_fax_data(data, year):
    '''
    data parsing function for fax data
    '''
    data_ = data.rename(columns={'Case Number': 'C_num'})
    used_cols = ['C_num', 'EmpName', 'JobTitle', 'JobCode',
                 'CertCode', 'WageRateFrom', 'RatePer', 'PrevWage_1',
                 'PrevWagePer_1', 'WorkCity_1', 'WorkState_1']
    data_ = data_[used_cols]
    # add a 'year' col to represent the year
    data_['year'] = year
    data_['WageRateFrom'] = data_['WageRateFrom'].apply(str2num)
    data_['PrevWage_1'] = data_['PrevWage_1'].apply(str2num)
    data_['annual_wage'] = data_['WageRateFrom']*data_['RatePer'].apply(rp2num)
    data_['annual_prevailing_wage'] = data_[
        'PrevWage_1']*data_['PrevWagePer_1'].apply(rp2num)
    # change the columns' name as required and retain necessary cols
    col_trans_dict = {'C_num': 'case_number', 'EmpName': 'employer', 'JobTitle': 'job_title',
                      'JobCode': 'job_code', 'CertCode': 'approval_status',
                      'WorkCity_1': 'city', 'WorkState_1': 'state'}
    data_ = data_.rename(columns=col_trans_dict)
    # fax data don't contain some certain fields, set them to nan
    missing_cols = ['date', 'tot_foreign_positions',
                    'naic_code', 'soc_name', 'county']
    data_ = pd.concat([data_, pd.DataFrame(columns=missing_cols)], axis=1)
    return data_[retain_fields]

Then we apply the function `process_fax_data` to the raw data:

In [5]:
for year in tqdm_notebook(range(2002, 2007)):
    year_str = str(year)[-2:]
    globals()['fax_'+year_str] = process_fax_data(globals()
                                                  ['fax_'+year_str], year)

# Efile data 02-07

Next we load efile data from 2002 to 2007:

In [6]:
for year in tqdm_notebook(range(2002, 2008)):
    year_str = str(year)[-2:]
    globals()['efile_'+year_str] = pd.read_csv('dataset/h1b_efile_FY' +
                                               year_str+'.txt', encoding='latin-1')

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (25,29,33,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (28,30,31,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (16,26,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


For efile data:

- The fields `tot_foreign_positions`, `naic_code`, `soc_name`, `county` are missing, so we set the values of these fields to be `np.nan`

- The fields `date`, `case_number`, `employer`, `job_title`, `job_code`, `approval_status`, `city`, `state` can be directly obtained from the raw data, we just need to modify their names

- The fields `annual_wage`, `annual_prevailing_wage` need to be calculated using wages and units. So, we need to convert them from string-type to numerical-type using function `str2num` and `rp2num`

- We convert the type of `date` from string to datetime

- We add a column `year` to represent the year of data source

- For year 2007, we need to do some additional work: we extract H1B data by specifying the `Program Designation` to be **R**. Also, we need to drop two columns.


In [7]:
efile_cols = efile_02.columns


def str2num(x):
    '''
    change string-type number to float
    '''
    try:
        return float(x)
    except:
        x = x.replace(',', '')
        if x[0] == '$':
            x = x[1:]
        try:
            return float(x)
        except:
            return np.nan


def rp2num(x):
    '''
    change RatePer to frequency number
    '''
    trans_dict = {'Year': 1, 'Month': 12,
                  '2 weeks': 26, 'Week': 52, 'Hour': 2080}
    try:
        return trans_dict[x]
    except:
        return np.nan


def str2time(x):
    '''
    change string-type time to datetime
    '''
    try:
        return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    except:
        return x


def process_efile_data(data, year):
    '''
    data parsing function for efile data
    '''
    data_ = data.copy()
    # extra parsing for 07's data
    if year == 2007:
        data_ = data_[data['Program Designation'] == 'R']
        data_.drop(['Program Designation', 'Withdrawn'], axis=1, inplace=True)
        data_.columns = efile_cols
    used_cols = ['SUBMITTED_DATE', 'CASE_NO', 'NAME', 'CITY_1', 'STATE_1',
                 'JOB_TITLE', 'JOB_CODE', 'APPROVAL_STATUS', 'WAGE_RATE_1',
                 'RATE_PER_1', 'PREVAILING_WAGE_1']
    data_ = data_[used_cols]
    data_['year'] = year
    data_['SUBMITTED_DATE'] = data_['SUBMITTED_DATE'].apply(str2time)
    data_['WAGE_RATE_1'] = data_['WAGE_RATE_1'].apply(str2num)
    data_['PREVAILING_WAGE_1'] = data_['PREVAILING_WAGE_1'].apply(str2num)
    data_['annual_wage'] = data_['WAGE_RATE_1'] * \
        data_['RATE_PER_1'].apply(rp2num)
    data_['annual_prevailing_wage'] = data_[
        'PREVAILING_WAGE_1']*data_['RATE_PER_1'].apply(rp2num)
    # change the columns' name as required and retain necessary cols
    col_trans_dict = {'SUBMITTED_DATE': 'date', 'CASE_NO': 'case_number', 'NAME': 'employer', 'JOB_TITLE': 'job_title',
                      'JOB_CODE': 'job_code', 'APPROVAL_STATUS': 'approval_status',
                      'CITY_1': 'city', 'STATE_1': 'state'}
    data_ = data_.rename(columns=col_trans_dict)
    # fax data don't contain some certain fields, set them to nan
    missing_cols = ['tot_foreign_positions', 'naic_code', 'soc_name', 'county']
    data_ = pd.concat([data_, pd.DataFrame(columns=missing_cols)], axis=1)
    return data_[retain_fields]

Then we apply the function `process_efile_data` to the raw data:

In [8]:
for year in tqdm_notebook(range(2002,2008)):
    year_str = str(year)[-2:]
    globals()['efile_'+year_str] = process_efile_data(globals()['efile_'+year_str],year)

# H1B data 08-18 

For data between 08 and 18, there's only one source of data for each year (except 2009, in which we also also have a efile-source of data)

In [9]:
for year in tqdm_notebook(range(2008, 2019)):
    year_str = str(year)[-2:]
    globals()['h1b_'+year_str] = pd.read_csv('dataset/H-1B_FY'+year_str+'.csv')

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (22,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (14,15,16,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Anaconda3\lib\site-packages\IPython\co

## efile 09 h1b 08

For H1B data in 2008 and efile data in 2009, their formats are very similar to those of efile data 02-07, so we only need to do some simple parsing and apply the function `process_efile_data` to them：

In [10]:
efile_09 = pd.read_csv('dataset/H-1B_FY09_efile.csv')


def rp2num(x):
    '''
    change RatePer to frequency number
    '''
    trans_dict = {'YR': 1, 'MTH': 12, 'BI': 26, 'WK': 52, 'HR': 2080}
    try:
        return trans_dict[x]
    except:
        return np.nan


efile_09 = efile_09[efile_09['PROGRAM_DESIGNATION'] == 'R']
efile_09.drop(['PROGRAM_DESIGNATION', 'WITHDRAWN'], axis=1, inplace=True)
efile_09 = efile_09.rename(
    columns={'OCCUPATIONAL_CODE': 'JOB_CODE', 'EMPLOYER_NAME': 'NAME'})
# efile_09.columns = efile_cols
efile_09 = process_efile_data(efile_09, 2009)

In [11]:
def rp2num(x):
    '''
    change RatePer to frequency number
    '''
    trans_dict = {'yr':1, 'mth':12, 'bi':26, 'wk':52, 'hr':2080}
    try:
        return trans_dict[x]
    except:
        return np.nan

h1b_08 = h1b_08[h1b_08['PROGRAM']=='R']
h1b_08 = h1b_08.drop(['PROGRAM','WITHDRAWN','OCCUPATIONAL_TITLE'],axis=1)
h1b_08 = process_efile_data(h1b_08, 2008)

## h1b 09-18

For H1B data in the periods of 2009-2018:
- we extract H1B data by specifying the column `VISA_CLASS` to be **H-1B**
- For year 2009-2014, the columns' names have significant differences from those of year 2015-2018, so we need to rename them. Also, the field `county` is missing, set it to `np.nan`
- For year 2015, the `wage` field is in the format of `"a number-a number"`. we need to deal with it using the function `range2num`
- The fields `annual_wage`, `annual_prevailing_wage` need to be calculated using wages and units. So, we need to convert wages and units from string-type to numerical-type using function `str2num` and `rp2num`
- We convert the type of `date` from string to datetime using the function `str2time`
- We add a column `year` to represent the year of data source


In [12]:
def str2num(x):
    '''
    change string-type number to float
    '''    
    try:
        return float(x)
    except:
        x = x.replace(',','')
        if x[0] == '$':
            x = x[1:]
        try:
            return float(x)
        except:
            return np.nan
    
def rp2num(x):
    '''
    change RatePer to frequency number
    '''
    trans_dict = {'Year':1, 'Month':12, 'Bi-Weekly':26, 'Week':52,'Hour':2080}
    try:
        return trans_dict[x]
    except:
        return np.nan
    
def str2time(x):
    '''
    change string-type time to datetime
    '''
    try:
        return datetime.strptime(x,'%Y/%m/%d')
    except:
        return x

def range2num(x):
    '''
    change wage range to number
    '''
    try:
        return float(x.split('-')[0].strip())
    except:
        return np.nan

cols_mod_dict = {
    'LCA_CASE_SUBMIT': 'CASE_SUBMITTED',
    'LCA_CASE_NUMBER': 'CASE_NUMBER',
    'LCA_CASE_EMPLOYER_NAME': 'EMPLOYER_NAME',
    'LCA_CASE_NAICS_CODE': 'NAICS_CODE',
    'LCA_CASE_JOB_TITLE': 'JOB_TITLE',
    'LCA_CASE_SOC_CODE': 'SOC_CODE',
    'LCA_CASE_SOC_NAME': 'SOC_NAME',
    'STATUS': 'CASE_STATUS',
    'LCA_CASE_WAGE_RATE_FROM': 'WAGE_RATE_OF_PAY_FROM',
    'LCA_CASE_WAGE_RATE_UNIT': 'WAGE_UNIT_OF_PAY',
    'PW_1': 'PREVAILING_WAGE',
    'PW_UNIT_1': 'PW_UNIT_OF_PAY',
    'LCA_CASE_WORKLOC1_CITY': 'WORKSITE_CITY',
    'LCA_CASE_WORKLOC1_STATE': 'WORKSITE_STATE'
}


def process_h1b_data(data, year):
    '''
    data parsing function for h1b data from 09 to 18
    '''
    data_ = data.copy()
    # extra parsing for 07's data
    if year in range(2009,2015):
        if year == 2010:
            data_['LCA_CASE_WAGE_RATE_UNIT'] = data_['PW_UNIT_1']
            data_ = data_.rename(columns={'WORK_LOCATION_CITY1':'WORKSITE_CITY', 'WORK_LOCATION_STATE1':'WORKSITE_STATE'})
        data_ = data_.rename(columns=cols_mod_dict)
        data_['WORKSITE_COUNTY'] = np.nan
    elif year==2015:
        data_ = data_.rename(columns={'TOTAL WORKERS':'TOTAL_WORKERS','NAIC_CODE':'NAICS_CODE'})
        data_['WAGE_RATE_OF_PAY_FROM'] = data_['WAGE_RATE_OF_PAY'].apply(range2num)
    elif year==2016:
        data_ = data_.rename(columns={'NAIC_CODE':'NAICS_CODE'})   
    # choose the samples of H1B
    try:
        data_ = data_[data_['VISA_CLASS']=='H-1B']
    except:
        pass
    
    used_cols = ['CASE_SUBMITTED', 'CASE_NUMBER', 'EMPLOYER_NAME','TOTAL_WORKERS', 'NAICS_CODE', 'JOB_TITLE',
                 'SOC_CODE', 'SOC_NAME', 'CASE_STATUS', 'WAGE_RATE_OF_PAY_FROM', 'WAGE_UNIT_OF_PAY', 'PREVAILING_WAGE',
                 'PW_UNIT_OF_PAY', 'WORKSITE_CITY', 'WORKSITE_STATE', 'WORKSITE_COUNTY']
    data_ = data_[used_cols]
    data_['year'] = year
    data_['CASE_SUBMITTED'] = data_['CASE_SUBMITTED'].apply(str2time)
    data_['WAGE_RATE_OF_PAY_FROM'] = data_['WAGE_RATE_OF_PAY_FROM'].apply(str2num)
    data_['PREVAILING_WAGE'] = data_['PREVAILING_WAGE'].apply(str2num)
    data_['annual_wage'] = data_['WAGE_RATE_OF_PAY_FROM'] * data_['WAGE_UNIT_OF_PAY'].apply(rp2num)
    data_['annual_prevailing_wage'] = data_['PREVAILING_WAGE']*data_['PW_UNIT_OF_PAY'].apply(rp2num)
    # change the columns' name as required and retain necessary cols
    col_trans_dict = {'CASE_SUBMITTED': 'date', 'CASE_NUMBER': 'case_number', 'EMPLOYER_NAME': 'employer',
                      'TOTAL_WORKERS':'tot_foreign_positions', 'NAICS_CODE':'naic_code', 'JOB_TITLE': 'job_title',
                      'SOC_CODE': 'job_code', 'SOC_NAME': 'soc_name', 'CASE_STATUS': 'approval_status',
                      'WORKSITE_CITY': 'city', 'WORKSITE_STATE': 'state','WORKSITE_COUNTY':'county'}
    data_ = data_.rename(columns=col_trans_dict)
    return data_[retain_fields]

We apply the function `process_h1b_data` to data in the periods of 2009-2018:

In [13]:
for year in tqdm_notebook(range(2009,2019)):
    globals()['h1b_'+str(year)[-2:]] = process_h1b_data(globals()['h1b_'+str(year)[-2:]],year)

Finally, we concatenate the dataframes for years from 2008 to 2018 and obtain the dataframe `h1b_0818`

In [14]:
h1b_0818 = pd.concat([h1b_08,efile_09]+[globals()['h1b_'+str(x)[-2:]] for x in range(2009,2019)], ignore_index=True)

Save the dataframe to `h1b_0818.pkl`:

In [15]:
h1b_0818.to_pickle('dataset/h1b_0818.pkl')

Also, we concatenate all the dataframes for years in all periods and obtain the dataframe `h1b_all`

In [16]:
fax_list = [globals()['fax_'+str(x)[-2:]] for x in range(2002,2007)]
efile_list = [globals()['efile_'+str(x)[-2:]] for x in range(2002,2008)]
h1b_all = pd.concat(fax_list+efile_list+[h1b_0818], ignore_index=True)

Save the dataframe to `h1b_all.pkl`:

In [17]:
h1b_all.to_pickle('dataset/h1b_all.pkl')